In [1]:
import sys

import numpy as np
import math
from scipy.optimize import fsolve

# python3 -m pip install pyopengl
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

# python3 -m pip install pygame
import pygame
from pygame.locals import *

from draw_object import ConfigureEnv

pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class CartPoleState(object):
    def __init__(self):   
        self.x=0.
        self.x1=0.
        self.th=.2; 
        self.th1=0. 

        # init constants
        self.tau = 1/1000.0; # with 1/1000 is better
        Mp = 1
        Mc = 1
        self.l = 1
        self.c1 = 1/(Mp+Mc)
        self.c2 = self.l*Mp/(Mp+Mc)
        self.g = 9.8

        # change the dynamicsNoise here
        self.dynamicsNoise = 0.
    
    def step(self, u):
        the2 = self.g*math.sin(self.th) + math.cos(self.th)*(-self.c1*u-self.c2*self.th1*self.th1*math.sin(self.th));
        the2 /= self.l*4/3 - self.c2*math.cos(self.th)*math.cos(self.th);
        x2 = self.c1*u + self.c2*(self.th1*self.th1*math.sin(self.th) - the2*math.cos(self.th));
        
        self.x   += self.tau*self.x1;
        self.x1  += self.tau*x2;
        self.th  += self.tau*self.th1;
        self.th1 += self.tau*the2;
        
        if self.dynamicsNoise:
            self.x1 += self.dynamicsNoise*np.random.normal(0,1);
            self.th1 += self.dynamicsNoise*np.random.normal(0,1);


In [3]:
def draw_objects(env, x, theta):
    # draw guide lines
    env.draw_line()  

    # cart
    glColor3f(1,1,1); 
    glPushMatrix()
    glTranslatef(x, 0., 0.)
    env.draw_cube(1., 0.2, 0.2)  
    env.draw_line2()
    
    # pole
    glColor3f(1,0,0); 
    glPushMatrix()
    glRotatef(theta*180./math.pi, 0., 1., 0.)
    glTranslatef(0, 0., 0.5)
    env.draw_cube(0.1, 0.1, 1.)  
    
    glPopMatrix()
    glPopMatrix()
    

In [4]:
def testMove(s, w):
    #
    u = w[0]*s.x + w[1]*s.x1 + w[2]*s.th + w[3]*s.th1;
    s.step(u)
    
    # return translation x and rotation theta
    return s.x, s.th

In [5]:
def computeW(l1, l2, l3, l4):
    # initialize w
    w = np.zeros(4)
    #0.8w2-0.6w4 = l1+..+l4 = tr(A~)
    #0.8w1*(11.76-0.6w3) + (0.8w3-5.88)*0.6w1 = l1*..*l4 = det(A~)
    def delta1(ws, l):
        return (0.6 * ws[3] + l) * l - (11.76 - 0.6 * ws[2])
    def delta2(ws, l):
        return 0.8 * ws[2] - 5.88 + l * 0.8 * ws[3]
    def func(ws, l):
        return 0.8 * ws[0] * delta1(ws, l) - 0.6 * ws[0] * delta2(ws, l)\
               + l * (0.8 * w[1] - l) * delta1(ws, l) \
               - l * 0.6 * ws[1] * delta2(ws,l)

    def fun(ws):
        return [ func(ws,l1), func(ws, l2), func(ws, l3), func(ws,l4)]
    root = fsolve(fun, np.array([1,2.6,53,16]), full_output=False)
    for i in range(5):
        root_ = fsolve(fun, root)
        root = root_
    print(root)
    w = np.array(root)
    
    return w

In [6]:
def main():
    pygame.init()
 
    display = (1000,750)
    pygame.display.set_mode(display, DOUBLEBUF|OPENGL, RESIZABLE)

    glRotatef(-90.,1.,0, 0)
    glScaled(0.8, 0.8, 0.8);

    env = ConfigureEnv()
    s = CartPoleState()
    
    # set eigenvalues
    lambda1 = -1
    lambda2 = -1
    lambda3 = -1
    lambda4 = -1
    # tr(A)^2 - 4 * det(A) == 0
    def constraint(l):
        return [l[0] + 10, l[1] + 3, np.sum(l) + 24, pow(np.sum(l), 2) - 4 * np.prod(l)]
    ls = fsolve(constraint, np.array([-1,-.5,-2,0]))
    print(f'lambdas should be {ls}')

    w = computeW(ls[0], ls[1], ls[2], ls[3])
    print(w)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
            env.mouseMove(event);

        glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
        
        # calculate and update opengl
        x, th = testMove(s, w)
        draw_objects(env, x, th)
        
        pygame.display.flip()
        pygame.time.wait(10)

main()

lambdas should be [-10.         -3.        -10.5447993  -0.4552007]
[ 24.48979592   3.63434546 232.28733979  40.        ]
[ 24.48979592   3.63434546 232.28733979  40.        ]


error: video system not initialized